# PyKoSpacing

- 띄어쓰기가 되어있지 않은 문장을 띄어쓰기 한 문장으로 변환

In [44]:
# pip install git+https://github.com/haven-jeon/PyKoSpacing.git

In [45]:
sent = '김철수는 극중 두 인격의 사나이 이광수 역을 맡았다. 철수는 한국 유일의 태권도 전승자를 가리는 결전의 날을 앞두고 10년간 함께 훈련한 사형인 유연재(김광수 분)를 찾으러 속세로 내려온 인물이다.'
new_sent = sent.replace(" ", '') # 띄어쓰기가 없는 문장 임의로 만들기
print(new_sent)

김철수는극중두인격의사나이이광수역을맡았다.철수는한국유일의태권도전승자를가리는결전의날을앞두고10년간함께훈련한사형인유연재(김광수분)를찾으러속세로내려온인물이다.


In [46]:
from pykospacing import Spacing
spacing = Spacing()
kospacing_sent = spacing(new_sent) 

print(sent)
print(kospacing_sent)

Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB

김철수는 극중 두 인격의 사나이 이광수 역을 맡았다. 철수는 한국 유일의 태권도 전승자를 가리는 결전의 날을 앞두고 10년간 함께 훈련한 사형인 유연재(김광수 분)를 찾으러 속세로 내려온 인물이다.
김철수는 극중 두 인격의 사나이 이광수 역을 맡았다. 철수는 한국 유일의 태권도 전승자를 가리는 결전의 날을 앞두고 10년간 함께 훈련한 사형인 유연재(김광수 분)를 찾으러 속세로 내려온 인물이다.


# Py-Hanspell

- 네이버 맞춤법 검사기 기반 맞춤법, 띄어쓰기 보정

In [47]:
# !pip install git+https://github.com/Seokhyeon-Park/hanspell.git

In [49]:
from hanspell import spell_checker

sent = "맞춤법틀리면외않되? 쓰고싶은대로쓰면돼지 "
spelled_sent = spell_checker.check(sent)

hanspell_sent = spelled_sent.checked
print(hanspell_sent)

맞춤법 틀리면 왜 안돼? 쓰고 싶은 대로 쓰면 되지 


# SoyNLP

- 단어 토크나이저 (품사 태깅, 단어 토큰화 등 지원)

In [50]:
# !pip install soynlp

### 1. 기존 형태소 분석기

In [51]:
from konlpy.tag import Okt
tokenizer = Okt()
print(tokenizer.morphs('에이비식스 이대휘 1월 최애돌 기부 요정'))
print(tokenizer.morphs('크러쉬 원더러스트 명반이다'))
print(tokenizer.morphs('메연이는 딥러닝 공부 중'))

['에이', '비식스', '이대', '휘', '1월', '최애', '돌', '기부', '요정']
['크러쉬', '원더', '러스트', '명반', '이다']
['메', '연', '이', '는', '딥', '러닝', '공부', '중']


### 2. 학습하기

In [52]:
import urllib.request
from soynlp import DoublespaceLineCorpus
from soynlp.word import WordExtractor

urllib.request.urlretrieve("https://raw.githubusercontent.com/lovit/soynlp/master/tutorials/2016-10-20.txt", filename="2016-10-20.txt")

('2016-10-20.txt', <http.client.HTTPMessage at 0x2d95cb5e0>)

In [53]:
# 훈련 데이터를 다수의 문서로 분리
corpus = DoublespaceLineCorpus("2016-10-20.txt")
len(corpus)

30091

In [54]:
# 학습하기
word_extractor = WordExtractor()
word_extractor.train(corpus)
word_score_table = word_extractor.extract()

training was done. used memory 1.690 Gb
all cohesion probabilities was computed. # words = 223348
all branching entropies was computed # words = 361598
all accessor variety was computed # words = 361598


### 3. SOYNLP의 응집 확률 cohesion probability

In [55]:
print(word_score_table["반포한"].cohesion_forward)
print(word_score_table["반포한강"].cohesion_forward)
print(word_score_table["반포한강공원"].cohesion_forward)
print(word_score_table["반포한강공원에"].cohesion_forward)
# 응집 확률이 가장 높은 반포한강공원이 하나의 단어

0.08838002913645132
0.19841268168224552
0.37891487632839754
0.33492963377557666


### 4. SOYNLP의 브랜칭 엔트로피 branching entropy

In [56]:
print(word_score_table["디스"].right_branching_entropy)
print(word_score_table["디스플"].right_branching_entropy)
print(word_score_table["디스플레"].right_branching_entropy)
print(word_score_table["디스플레이"].right_branching_entropy) # 다시 증가 -> 아 이게 단어 !!

1.6371694761537934
-0.0
-0.0
3.1400392861792916


### 5. SOYNLP의 L tokenizer

In [57]:
from soynlp.tokenizer import LTokenizer

scores = {word:score.cohesion_forward for word, score in word_score_table.items()}
l_tokenizer = LTokenizer(scores=scores)
l_tokenizer.tokenize("국제사회와 우리의 노력들로 범죄를 척결하자", flatten=False)

[('국제사회', '와'), ('우리', '의'), ('노력', '들로'), ('범죄', '를'), ('척결', '하자')]

### 6. 최대 점수 토크나이저

In [58]:
from soynlp.tokenizer import MaxScoreTokenizer

maxscore_tokenizer = MaxScoreTokenizer(scores=scores)
maxscore_tokenizer.tokenize("국제사회와우리의노력들로범죄를척결하자")

['국제사회', '와', '우리', '의', '노력', '들로', '범죄', '를', '척결', '하자']

### 7. SOYNLP 반복되는 문자 정제

In [59]:
from soynlp.normalizer import *

print(emoticon_normalize('앜ㅋㅋㅋㅋ이영화존잼쓰ㅠㅠㅠㅠㅠ', num_repeats=2))
print(emoticon_normalize('앜ㅋㅋㅋㅋㅋㅋㅋㅋㅋ이영화존잼쓰ㅠㅠㅠㅠ', num_repeats=2))
print(emoticon_normalize('앜ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ이영화존잼쓰ㅠㅠㅠㅠㅠㅠ', num_repeats=2))
print(emoticon_normalize('앜ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ이영화존잼쓰ㅠㅠㅠㅠㅠㅠㅠㅠ', num_repeats=2))

아ㅋㅋ영화존잼쓰ㅠㅠ
아ㅋㅋ영화존잼쓰ㅠㅠ
아ㅋㅋ영화존잼쓰ㅠㅠ
아ㅋㅋ영화존잼쓰ㅠㅠ


In [60]:
print(repeat_normalize('와하하하하하하하하하핫', num_repeats=2))
print(repeat_normalize('와하하하하하하핫', num_repeats=2))
print(repeat_normalize('와하하하하핫', num_repeats=2))

와하하핫
와하하핫
와하하핫


# Customized KoNLPy

In [61]:
# !pip install customized_konlpy

In [62]:
from ckonlpy.tag import Twitter
twitter = Twitter()
twitter.morphs('메연이는 딥러닝 공부중이에요.')

/opt/homebrew/Caskroom/miniforge/base/envs/yeonii/lib/python3.8/site-packages/konlpy/tag/_okt.py:17: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


['메', '연', '이', '는', '딥', '러닝', '공부', '중이', '에요', '.']

In [63]:
twitter.add_dictionary('메연이', 'Noun')
twitter.add_dictionary('딥러닝', 'Noun')

In [64]:
twitter.morphs('메연이는 딥러닝 공부중이에요.')

['메연이', '는', '딥러닝', '공부', '중이', '에요', '.']

# KSS

In [65]:
import mecab_ko_dic
import kss

text_kor1 = '딥 러닝 자연어 처리가 재미있기는 합니다. 그런데 문제는 영어보다 한국어로 할 때 너무 어렵습니다. 이제 해보면 알걸요?'

print('한국어 문장 토큰화 :',kss.split_sentences(text_kor1))

한국어 문장 토큰화 : ['딥 러닝 자연어 처리가 재미있기는 합니다.', '그런데 문제는 영어보다 한국어로 할 때 너무 어렵습니다.', '이제 해보면 알걸요?']


# KoNLPy

In [66]:
from konlpy.tag import Kkma
kkma = Kkma()

text_kor2 = "같은 날 피고인 2에게 그중 여행용 가방과 현금 1억 9,600만 원을 전달하였다."

print('꼬꼬마 형태소 분석 :',kkma.morphs(text_kor2))
print('꼬꼬마 품사 태깅 :',kkma.pos(text_kor2))
print('꼬꼬마 명사 추출 :',kkma.nouns(text_kor2))  

꼬꼬마 형태소 분석 : ['같', '은', '날', '피고인', '2', '에게', '그중', '여행용', '가방', '과', '현금', '1', '억', '9,600', '만', '원', '을', '전달', '하', '였', '다', '.']
꼬꼬마 품사 태깅 : [('같', 'VA'), ('은', 'ETD'), ('날', 'NNG'), ('피고인', 'NNG'), ('2', 'NR'), ('에게', 'JKM'), ('그중', 'NNG'), ('여행용', 'NNG'), ('가방', 'NNG'), ('과', 'JKM'), ('현금', 'NNG'), ('1', 'NR'), ('억', 'NR'), ('9,600', 'NR'), ('만', 'NR'), ('원', 'NNM'), ('을', 'JKO'), ('전달', 'NNG'), ('하', 'XSV'), ('였', 'EPT'), ('다', 'EFN'), ('.', 'SF')]
꼬꼬마 명사 추출 : ['날', '피고인', '2', '그중', '여행용', '가방', '현금', '1', '1억', '억', '9,600', '9,600만', '만', '원', '전달']


In [67]:
from konlpy.tag import Okt
okt = Okt()

print('OKT 형태소 분석 :',okt.morphs(text_kor2))
print('OKT 품사 태깅 :',okt.pos(text_kor2))
print('OKT 명사 추출 :',okt.nouns(text_kor2)) 

OKT 형태소 분석 : ['같은', '날', '피고인', '2', '에게', '그중', '여행', '용', '가방', '과', '현금', '1억', '9,600만', '원', '을', '전달', '하였다', '.']
OKT 품사 태깅 : [('같은', 'Adjective'), ('날', 'Noun'), ('피고인', 'Noun'), ('2', 'Number'), ('에게', 'Josa'), ('그중', 'Adverb'), ('여행', 'Noun'), ('용', 'Noun'), ('가방', 'Noun'), ('과', 'Josa'), ('현금', 'Noun'), ('1억', 'Number'), ('9,600만', 'Number'), ('원', 'Noun'), ('을', 'Josa'), ('전달', 'Noun'), ('하였다', 'Verb'), ('.', 'Punctuation')]
OKT 명사 추출 : ['날', '피고인', '여행', '용', '가방', '현금', '원', '전달']


In [68]:
from konlpy.tag import Komoran
komoran = Komoran()

print('Komoran 형태소 분석 :',komoran.morphs(text_kor2))
print('Komoran 품사 태깅 :',komoran.pos(text_kor2))
print('Komoran 명사 추출 :',komoran.nouns(text_kor2)) 

Komoran 형태소 분석 : ['같', '은', '날', '피고인', '2', '에게', '그중', '여행', '용', '가방', '과', '현금', '1', '억', '9', ',', '600', '만', '원', '을', '전달', '하', '았', '다', '.']
Komoran 품사 태깅 : [('같', 'VA'), ('은', 'ETM'), ('날', 'NNG'), ('피고인', 'NNP'), ('2', 'SN'), ('에게', 'JKB'), ('그중', 'NNG'), ('여행', 'NNG'), ('용', 'XSN'), ('가방', 'NNG'), ('과', 'JC'), ('현금', 'NNP'), ('1', 'SN'), ('억', 'NR'), ('9', 'SN'), (',', 'SP'), ('600', 'SN'), ('만', 'NR'), ('원', 'NNB'), ('을', 'JKO'), ('전달', 'NNG'), ('하', 'XSV'), ('았', 'EP'), ('다', 'EF'), ('.', 'SF')]
Komoran 명사 추출 : ['날', '피고인', '그중', '여행', '가방', '현금', '원', '전달']


In [69]:
from konlpy.tag import Hannanum
hannanum = Hannanum()

print('Hannanum 형태소 분석 :',hannanum.morphs(text_kor2))
print('Hannanum 품사 태깅 :',hannanum.pos(text_kor2))
print('Hannanum 명사 추출 :',hannanum.nouns(text_kor2)) 

Hannanum 형태소 분석 : ['같', '은', '나', 'ㄹ', '피고', '이', 'ㄴ', '2', '에게', '그중', '여행용', '가방', '과', '현금', '1억', '9,600', '만', '원', '을', '전달', '하', '었다', '.']
Hannanum 품사 태깅 : [('같', 'P'), ('은', 'E'), ('나', 'P'), ('ㄹ', 'E'), ('피고', 'N'), ('이', 'J'), ('ㄴ', 'E'), ('2', 'N'), ('에게', 'J'), ('그중', 'M'), ('여행용', 'N'), ('가방', 'N'), ('과', 'J'), ('현금', 'N'), ('1억', 'N'), ('9,600', 'N'), ('만', 'J'), ('원', 'N'), ('을', 'J'), ('전달', 'N'), ('하', 'X'), ('었다', 'E'), ('.', 'S')]
Hannanum 명사 추출 : ['피고', '2', '여행용', '가방', '현금', '1억', '9,600', '원', '전달']
